In [48]:
import requests
import pandas as pd

# URL de base para a API de livros
base_url = "https://openlibrary.org/search.json"

# Parametros da requisição de acordo com documentação da API e necessidade do usuário
params = {
    "q": "(title:harry potter OR title:lords of the rings OR author:jane austen)",
    "fields": "title, publish_year, author_key, author_name, subject",
    "sort": "rating",
    "limit": 1000
}

# Faz a requisição GET para a API com os parâmetros definidos
response = requests.get(base_url, params=params)

if response.status_code == 200: # Verifica se a requisição foi bem-sucedida, cód 200 padrão HTTP
    data = response.json() # Converte a resposta JSON em um dicionário Python
    livros = data["docs"] # Cria objeto livros que contém apenas os dados que estão dentro da chave docs da resposta da API.
    df_livros = pd.DataFrame(livros) # Cria um DataFrame pandas a partir da lista de livros
    print(df_livros.head()) # Exibe as primeiras linhas do DataFrame com as colunas especificadas no fields
else:
    print(f"Erro na requisição: {response.status_code}")

print(response.status_code)
print(response.text)

               author_key                     author_name  \
0              [OL26320A]                [J.R.R. Tolkien]   
1              [OL26320A]                [J.R.R. Tolkien]   
2              [OL26320A]                [J.R.R. Tolkien]   
3              [OL23919A]                 [J. K. Rowling]   
4  [OL23919A, OL2703953A]  [J. K. Rowling, Mary GrandPré]   

                                        publish_year  \
0  [1954, 1956, 1959, 1963, 1965, 1966, 1967, 196...   
1  [1954, 1965, 1966, 1967, 1968, 1969, 1970, 197...   
2  [1954, 1955, 1960, 1962, 1963, 1965, 1966, 196...   
3  [2016, 2018, 1999, 2004, 2007, 2008, 2009, 201...   
4  [2016, 2017, 2018, 2020, 2021, 2023, 2005, 200...   

                                    title  \
0              The Fellowship of the Ring   
1                   The Lord of the Rings   
2                          The Two Towers   
3               Harry Potter (series) 1-7   
4  Harry Potter and the Half-Blood Prince   

                         

Tratamento dos dados

1. Entendendo os dados

In [49]:
df_livros.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   author_key    876 non-null    object
 1   author_name   876 non-null    object
 2   publish_year  971 non-null    object
 3   title         1000 non-null   object
 4   subject       467 non-null    object
dtypes: object(5)
memory usage: 39.2+ KB


In [50]:
df_livros.columns

Index(['author_key', 'author_name', 'publish_year', 'title', 'subject'], dtype='object')

In [51]:
df_livros.shape

(1000, 5)

In [52]:
df_livros.isnull().sum()

author_key      124
author_name     124
publish_year     29
title             0
subject         533
dtype: int64

2. Normalizando os dados  
2.1 Padronizando nulos

In [53]:
print(df_livros[df_livros['subject'].isnull()])

                  author_key                    author_name  \
24              [OL8354734A]                 [SuperSummary]   
27   [OL13498939A, OL21594A]  [Rodolfo Chenel, Jane Austen]   
31                       NaN                            NaN   
37              [OL2853379A]             [Cedco Publishing]   
38              [OL3173981A]           [Barbara M. Zollner]   
..                       ...                            ...   
995             [OL9803131A]                [Nishat Rahman]   
996             [OL7441888A]                 [Martha Stone]   
997             [OL1300682A]                 [Thomas Brown]   
998   [OL9852462A, OL21594A]      [juan perez, Jane Austen]   
999             [OL9829754A]                [Taylor MAELEE]   

                             publish_year  \
24   [2017, 2018, 2019, 2020, 2021, 2022]   
27                                 [1899]   
31                                    NaN   
37                                 [2003]   
38                

In [54]:
df_livros['author_name'].fillna('Sem informação', inplace=True) # Preenche valores nulos com 'Sem informação'
df_livros.isnull().sum()

C:\Users\basil\AppData\Local\Temp\ipykernel_39048\2781930646.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_livros['author_name'].fillna('Sem informação', inplace=True) # Preenche valores nulos com 'Sem informação'


author_key      124
author_name       0
publish_year     29
title             0
subject         533
dtype: int64

In [55]:
df_livros['subject'].fillna('Sem informação', inplace=True) # Preenche valores nulos com 'Sem informação'
df_livros.isnull().sum()

C:\Users\basil\AppData\Local\Temp\ipykernel_39048\795722335.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_livros['subject'].fillna('Sem informação', inplace=True) # Preenche valores nulos com 'Sem informação'


author_key      124
author_name       0
publish_year     29
title             0
subject           0
dtype: int64

2.2 Padronizando author_name

In [56]:
# Verifica os tipos de dados na coluna author_name
df_livros['author_name'].apply(type).value_counts()

author_name
<class 'list'>    876
<class 'str'>     124
Name: count, dtype: int64

In [57]:
# Padroniza os nomes dos autores removendo acentuação e convertendo para minúsculas
!pip install unidecode
from unidecode import unidecode

def padronizando_autor(autor): # Função chamada de 'padronizando_autor' para padronizar o parâmetro 'autor'
    return unidecode(autor. lower()) # Função retorna o parâmetro 'autor' sem acentuação e em letras minúsculas

# Cria uma nova coluna 'autor_padronizado' aplicando a função 'padronizando_autor' em cada elemento da coluna 'author_name'
df_livros['autor_padronizado'] = df_livros['author_name'].apply(lambda x: [padronizando_autor(autor) for autor in x] if isinstance(x, list) else [])
df_livros.head()


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


,author_key,author_name,publish_year,title,subject,autor_padronizado
0,[OL26320A],[J.R.R. Tolkien],"[1954, 1956, 1959, 1963, 1965, 1966, 1967, 196...",The Fellowship of the Ring,"[Elves, Dwarves, evil, fear, hope, young adult...",[j.r.r. tolkien]
1,[OL26320A],[J.R.R. Tolkien],"[1954, 1965, 1966, 1967, 1968, 1969, 1970, 197...",The Lord of the Rings,"[The Lord of the Rings, Fiction, Ficción, Engl...",[j.r.r. tolkien]
2,[OL26320A],[J.R.R. Tolkien],"[1954, 1955, 1960, 1962, 1963, 1965, 1966, 196...",The Two Towers,"[Ents, Orcs, hobbits, magic, Ficción fantástic...",[j.r.r. tolkien]
3,[OL23919A],[J. K. Rowling],"[2016, 2018, 1999, 2004, 2007, 2008, 2009, 201...",Harry Potter (series) 1-7,"[New York Times bestseller, nyt:series_books=2...",[j. k. rowling]
4,"[OL23919A, OL2703953A]","[J. K. Rowling, Mary GrandPré]","[2016, 2017, 2018, 2020, 2021, 2023, 2005, 200...",Harry Potter and the Half-Blood Prince,"[orphans, foster homes, romans, magie, adolesc...","[j. k. rowling, mary grandpre]"


2.2.1 Criando lista de autores únicos

In [58]:
# Lista criada para incluir apenas os autores que estão em formato lista dentro do df_livros
lista_autores = df_livros['autor_padronizado'][df_livros['autor_padronizado'].apply(lambda x: isinstance(x, list))]

print(lista_autores[:5])

0                  [j.r.r. tolkien]
1                  [j.r.r. tolkien]
2                  [j.r.r. tolkien]
3                   [j. k. rowling]
4    [j. k. rowling, mary grandpre]
Name: autor_padronizado, dtype: object


In [59]:
# Transformando a série de listas em uma lista contínua de nomes de autores
lista_autores_continua = [] # Lista vazia para armazenar os autores
for lista in lista_autores: # Para cada lista presente na lista_autores
    lista_autores_continua.extend(lista) # Adicionar cada autor da lista à lista_autores_continua

print(lista_autores_continua[:5])

['j.r.r. tolkien', 'j.r.r. tolkien', 'j.r.r. tolkien', 'j. k. rowling', 'j. k. rowling']


In [60]:
# Cria um DataFrame com os autores únicos, ordenados alfabeticamente
df_autores_unicos = pd.DataFrame(sorted(set(lista_autores_continua)), columns=['autor'])
df_autores_unicos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 680 entries, 0 to 679
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   autor   680 non-null    object
dtypes: object(1)
memory usage: 5.4+ KB


In [61]:
# Verifica os tipos de dados na coluna autor do DataFrame df_autores_unicos
df_autores_unicos['autor'].apply(type).value_counts()

autor
<class 'str'>    680
Name: count, dtype: int64

In [62]:
df_livros.head()

,author_key,author_name,publish_year,title,subject,autor_padronizado
0,[OL26320A],[J.R.R. Tolkien],"[1954, 1956, 1959, 1963, 1965, 1966, 1967, 196...",The Fellowship of the Ring,"[Elves, Dwarves, evil, fear, hope, young adult...",[j.r.r. tolkien]
1,[OL26320A],[J.R.R. Tolkien],"[1954, 1965, 1966, 1967, 1968, 1969, 1970, 197...",The Lord of the Rings,"[The Lord of the Rings, Fiction, Ficción, Engl...",[j.r.r. tolkien]
2,[OL26320A],[J.R.R. Tolkien],"[1954, 1955, 1960, 1962, 1963, 1965, 1966, 196...",The Two Towers,"[Ents, Orcs, hobbits, magic, Ficción fantástic...",[j.r.r. tolkien]
3,[OL23919A],[J. K. Rowling],"[2016, 2018, 1999, 2004, 2007, 2008, 2009, 201...",Harry Potter (series) 1-7,"[New York Times bestseller, nyt:series_books=2...",[j. k. rowling]
4,"[OL23919A, OL2703953A]","[J. K. Rowling, Mary GrandPré]","[2016, 2017, 2018, 2020, 2021, 2023, 2005, 200...",Harry Potter and the Half-Blood Prince,"[orphans, foster homes, romans, magie, adolesc...","[j. k. rowling, mary grandpre]"


2.3 Padronizando subtitle

In [63]:
def padronizando_assunto(assunto): # Função chamada de 'padronizando_assunto' para padronizar o parâmetro 'assunto'
    return unidecode(assunto.lower()) # Função retorna o parâmetro 'assunto' sem acentuação e em letras minúsculas


# Cria uma nova coluna 'assunto_padronizado' aplicando a função 'padronizando_assunto' em cada elemento da coluna 'subject'
df_livros['assunto_padronizado'] = df_livros[ 'subject'].apply(lambda x: [padronizando_assunto(assunto) for assunto in x] if isinstance(x, list) else [])
df_livros.head()

,author_key,author_name,publish_year,title,subject,autor_padronizado,assunto_padronizado
0,[OL26320A],[J.R.R. Tolkien],"[1954, 1956, 1959, 1963, 1965, 1966, 1967, 196...",The Fellowship of the Ring,"[Elves, Dwarves, evil, fear, hope, young adult...",[j.r.r. tolkien],"[elves, dwarves, evil, fear, hope, young adult..."
1,[OL26320A],[J.R.R. Tolkien],"[1954, 1965, 1966, 1967, 1968, 1969, 1970, 197...",The Lord of the Rings,"[The Lord of the Rings, Fiction, Ficción, Engl...",[j.r.r. tolkien],"[the lord of the rings, fiction, ficcion, engl..."
2,[OL26320A],[J.R.R. Tolkien],"[1954, 1955, 1960, 1962, 1963, 1965, 1966, 196...",The Two Towers,"[Ents, Orcs, hobbits, magic, Ficción fantástic...",[j.r.r. tolkien],"[ents, orcs, hobbits, magic, ficcion fantastic..."
3,[OL23919A],[J. K. Rowling],"[2016, 2018, 1999, 2004, 2007, 2008, 2009, 201...",Harry Potter (series) 1-7,"[New York Times bestseller, nyt:series_books=2...",[j. k. rowling],"[new york times bestseller, nyt:series_books=2..."
4,"[OL23919A, OL2703953A]","[J. K. Rowling, Mary GrandPré]","[2016, 2017, 2018, 2020, 2021, 2023, 2005, 200...",Harry Potter and the Half-Blood Prince,"[orphans, foster homes, romans, magie, adolesc...","[j. k. rowling, mary grandpre]","[orphans, foster homes, romans, magie, adolesc..."


2.3.1 Criando lista de assuntos únicos

In [64]:
# Verifica se há algum valor que não seja lista
nao_lista = df_livros['assunto_padronizado'].apply(lambda x: not isinstance(x, list))

# Se houver pelo menos um valor que não seja lista, mostra as linhas
if nao_lista.any():
    print("Existem valores que não são listas.")
    print(df_livros.loc[nao_lista, 'assunto_padronizado'].head())
else:
    print("Todos os valores são listas.")

Todos os valores são listas.


In [65]:
# Lista criada para incluir apenas os assuntos que estão em formato lista dentro do df_livros
lista_assuntos = df_livros['assunto_padronizado'][df_livros['assunto_padronizado'].apply(lambda x: isinstance(x, list))]
print(lista_assuntos[:5])


0    [elves, dwarves, evil, fear, hope, young adult...
1    [the lord of the rings, fiction, ficcion, engl...
2    [ents, orcs, hobbits, magic, ficcion fantastic...
3    [new york times bestseller, nyt:series_books=2...
4    [orphans, foster homes, romans, magie, adolesc...
Name: assunto_padronizado, dtype: object


In [66]:
lista_assuntos_continua = [] # Lista vazia para armazenar os assuntos
for lista in lista_assuntos: # Para cada lista presente na lista_assuntos
    lista_assuntos_continua.extend(lista) # Adicionar cada assunto da lista à lista_assuntos_continua

print(lista_assuntos_continua[:5])

['elves', 'dwarves', 'evil', 'fear', 'hope']


In [67]:
df_assuntos_unicos = pd.DataFrame(sorted(set(lista_assuntos_continua)), columns=['assunto'])
df_assuntos_unicos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1343 entries, 0 to 1342
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   assunto  1343 non-null   object
dtypes: object(1)
memory usage: 10.6+ KB


In [68]:
df_assuntos_unicos['assunto'].apply(type).value_counts()

assunto
<class 'str'>    1343
Name: count, dtype: int64

2.4 Padronizando publish_year

In [69]:
lista_anos = df_livros['publish_year'][df_livros['publish_year'].apply(lambda x: isinstance(x, list))]
print(lista_anos[:5])

0    [1954, 1956, 1959, 1963, 1965, 1966, 1967, 196...
1    [1954, 1965, 1966, 1967, 1968, 1969, 1970, 197...
2    [1954, 1955, 1960, 1962, 1963, 1965, 1966, 196...
3    [2016, 2018, 1999, 2004, 2007, 2008, 2009, 201...
4    [2016, 2017, 2018, 2020, 2021, 2023, 2005, 200...
Name: publish_year, dtype: object


In [70]:
lista_anos_continua = [] # Lista vazia para armazenar os anos
for lista in lista_anos: # Para cada lista presente na lista_anos
    lista_anos_continua.extend(lista) # Adicionar cada ano da lista à lista_anos_continua

print(lista_anos_continua[:5])

[1954, 1956, 1959, 1963, 1965]


In [71]:
df_anos_unicos = pd.DataFrame(sorted((set(lista_anos_continua))), columns=['ano'])
df_anos_unicos.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169 entries, 0 to 168
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   ano     169 non-null    int64
dtypes: int64(1)
memory usage: 1.4 KB


In [72]:
df_anos_unicos['ano'].apply(type).value_counts()

ano
<class 'int'>    169
Name: count, dtype: int64

2.5 Padronizando title

In [73]:
def padronizando_titulo(titulo):
    return unidecode(titulo.lower())

df_livros['titulo_padronizado'] = df_livros['title'].apply(padronizando_titulo)
df_livros.head()

,author_key,author_name,publish_year,title,subject,autor_padronizado,assunto_padronizado,titulo_padronizado
0,[OL26320A],[J.R.R. Tolkien],"[1954, 1956, 1959, 1963, 1965, 1966, 1967, 196...",The Fellowship of the Ring,"[Elves, Dwarves, evil, fear, hope, young adult...",[j.r.r. tolkien],"[elves, dwarves, evil, fear, hope, young adult...",the fellowship of the ring
1,[OL26320A],[J.R.R. Tolkien],"[1954, 1965, 1966, 1967, 1968, 1969, 1970, 197...",The Lord of the Rings,"[The Lord of the Rings, Fiction, Ficción, Engl...",[j.r.r. tolkien],"[the lord of the rings, fiction, ficcion, engl...",the lord of the rings
2,[OL26320A],[J.R.R. Tolkien],"[1954, 1955, 1960, 1962, 1963, 1965, 1966, 196...",The Two Towers,"[Ents, Orcs, hobbits, magic, Ficción fantástic...",[j.r.r. tolkien],"[ents, orcs, hobbits, magic, ficcion fantastic...",the two towers
3,[OL23919A],[J. K. Rowling],"[2016, 2018, 1999, 2004, 2007, 2008, 2009, 201...",Harry Potter (series) 1-7,"[New York Times bestseller, nyt:series_books=2...",[j. k. rowling],"[new york times bestseller, nyt:series_books=2...",harry potter (series) 1-7
4,"[OL23919A, OL2703953A]","[J. K. Rowling, Mary GrandPré]","[2016, 2017, 2018, 2020, 2021, 2023, 2005, 200...",Harry Potter and the Half-Blood Prince,"[orphans, foster homes, romans, magie, adolesc...","[j. k. rowling, mary grandpre]","[orphans, foster homes, romans, magie, adolesc...",harry potter and the half-blood prince


In [74]:
df_livros['titulo_padronizado'].apply(type).value_counts()

titulo_padronizado
<class 'str'>    1000
Name: count, dtype: int64

In [75]:
df_titulos_unicos = pd.DataFrame(sorted(set(df_livros['titulo_padronizado'])), columns=['titulo'])
df_titulos_unicos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 874 entries, 0 to 873
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   titulo  874 non-null    object
dtypes: object(1)
memory usage: 7.0+ KB


Transformando listas em registros únicos por linha (autor_padronizado, publish_year e assunto_padronizado)

In [76]:
df_livros['assunto_padronizado'].apply(lambda x: isinstance(x, list)).value_counts()

assunto_padronizado
True    1000
Name: count, dtype: int64

In [77]:
df_livros['publish_year'].apply(lambda x: isinstance(x, list)).value_counts()

publish_year
True     971
False     29
Name: count, dtype: int64

In [78]:
df_livros['autor_padronizado'].apply(lambda x: isinstance(x, list)).value_counts()

autor_padronizado
True    1000
Name: count, dtype: int64

In [79]:
df_livros_exploded = df_livros.explode('autor_padronizado')
df_livros_exploded = df_livros_exploded.explode('publish_year')
df_livros_exploded = df_livros_exploded.explode('assunto_padronizado')
df_livros_exploded.head()

,author_key,author_name,publish_year,title,subject,autor_padronizado,assunto_padronizado,titulo_padronizado
0,[OL26320A],[J.R.R. Tolkien],1954,The Fellowship of the Ring,"[Elves, Dwarves, evil, fear, hope, young adult...",j.r.r. tolkien,elves,the fellowship of the ring
0,[OL26320A],[J.R.R. Tolkien],1954,The Fellowship of the Ring,"[Elves, Dwarves, evil, fear, hope, young adult...",j.r.r. tolkien,dwarves,the fellowship of the ring
0,[OL26320A],[J.R.R. Tolkien],1954,The Fellowship of the Ring,"[Elves, Dwarves, evil, fear, hope, young adult...",j.r.r. tolkien,evil,the fellowship of the ring
0,[OL26320A],[J.R.R. Tolkien],1954,The Fellowship of the Ring,"[Elves, Dwarves, evil, fear, hope, young adult...",j.r.r. tolkien,fear,the fellowship of the ring
0,[OL26320A],[J.R.R. Tolkien],1954,The Fellowship of the Ring,"[Elves, Dwarves, evil, fear, hope, young adult...",j.r.r. tolkien,hope,the fellowship of the ring


Removendo colunas que não serão utilizadas no sql

In [80]:
print(df_livros_exploded.columns.tolist())


['author_key', 'author_name', 'publish_year', 'title', 'subject', 'autor_padronizado', 'assunto_padronizado', 'titulo_padronizado']


In [81]:
df_livros_exploded = df_livros_exploded.drop(columns=['author_key', 'author_name', 'title', 'subject'])

In [82]:
print(df_livros_exploded.columns.tolist())

['publish_year', 'autor_padronizado', 'assunto_padronizado', 'titulo_padronizado']


In [83]:
df_livros_exploded = df_livros_exploded[['titulo_padronizado', 'autor_padronizado', 'assunto_padronizado', 'publish_year', ]]

In [84]:
df_livros_exploded = df_livros_exploded.rename(columns={'publish_year': 'ano_publicacao'})
df_livros_exploded.head()

,titulo_padronizado,autor_padronizado,assunto_padronizado,ano_publicacao
0,the fellowship of the ring,j.r.r. tolkien,elves,1954
0,the fellowship of the ring,j.r.r. tolkien,dwarves,1954
0,the fellowship of the ring,j.r.r. tolkien,evil,1954
0,the fellowship of the ring,j.r.r. tolkien,fear,1954
0,the fellowship of the ring,j.r.r. tolkien,hope,1954


In [85]:
print(df_livros_exploded.columns.tolist())

['titulo_padronizado', 'autor_padronizado', 'assunto_padronizado', 'ano_publicacao']


3. Conexão com MySQL

In [86]:
!pip install sqlalchemy

from dotenv import load_dotenv
import os
from sqlalchemy import create_engine

# Carrega as variáveis do .env
load_dotenv()

# Lê as variáveis
usuario = os.getenv('MYSQL_USUARIO')
senha = os.getenv('MYSQL_SENHA')
host = os.getenv('MYSQL_HOST')
porta = os.getenv('MYSQL_PORTA')
banco = os.getenv('MYSQL_BANCO')

print("Usuário:", usuario)
print("Senha:", senha)
print("Host:", host)
print("Porta:", porta)
print("Banco:", banco)




[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Usuário: root
Senha: bar2016
Host: localhost
Porta: 3306
Banco: projeto_api_livros


In [87]:
# Cria a conexão
!pip install pymysql




[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [88]:
engine = create_engine(f'mysql+pymysql://{usuario}:{senha}@{host}:{porta}/{banco}')

In [90]:
    # Exporta df_livros_exploded
    df_livros_exploded.to_sql(name='livros', con=engine, if_exists='replace', index=False)

    # Exporta df_titulos_unicos
    df_titulos_unicos.to_sql(name='titulos_unicos', con=engine, if_exists='replace', index=False)

    # Exporta df_autores_unicos
    df_autores_unicos.to_sql(name='autores_unicos', con=engine, if_exists='replace', index=False)

    # Exporta df_anos_unicos
    df_anos_unicos.to_sql(name='anos_unicos', con=engine, if_exists='replace', index=False)

    # Exporta df_assuntos_unicos
    df_assuntos_unicos.to_sql(name='assuntos_unicos', con=engine, if_exists='replace', index=False)


    print("Todos os DataFrames foram exportados com sucesso para o MySQL!")

Todos os DataFrames foram exportados com sucesso para o MySQL!
